In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

In [2]:
#Adj R square
def adj_r2_score(model,X_test, y_test,):
    y_pred = model.predict(X_test)
    # model.coefs_ doesn't exist
    adj = 1 - float(len(y_test)-1)/(len(y_test)-model.n_features_-1) * \
            (1 - r2_score(y_test,y_pred))
    return adj

# Evalute random search
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    MSE = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
#    r2_adjusted = adj_r2_score(model, X_test, y_test)
    print('Model Validation')
    print('MSE: {0}'.format(MSE))
    print('R^2: {0}'.format(r2))
#    print('R^2 Adjusted: {0}'.format(r2_adjusted))

In [3]:
# read csv file
dataset = pd.read_csv('./csvfiles/output_training.csv')
dataset['merchanttype'] = dataset['merchanttype'].astype('category')
dataset['merchantname'] = dataset['merchantname'].astype('category')
dataset['topads'] = dataset['topads'].replace((1,0), ('yes', 'no'))
dataset['topads'] = dataset['topads'].astype('category')
# drop column 'id' and 'prodname'
dataset = dataset.drop(['id', 'prodname', 'merchantname', 'actualrevcount'], axis=1)
# some 'revpictotal' has 9999 value and most of them have
# 'actualrevcount' = 10, only 5a986371b8a9f712ce73da7f has 119 'actualrevcount'
# replace 9999 with np.nan
dataset['revpictotal'] = dataset['revpictotal'].replace(9999, np.nan)
# data imputation
# mean technique
meanway = dataset['revpictotal'].mean()
dataset['revpictotal'] = dataset['revpictotal'].fillna(meanway)

In [4]:
# result reproducibility
np.random.seed(42)

In [5]:
dataset.head()

,merchanttype,topads,cashback,cashbackval,price,prodrating,reviewcount,negreview,posreview,answerscnt,otheragreemean,ratingmosthelpful,possentiment,negsentiment,sentipolarity,reviewersrep,revpictotal,prodpicstotal,ranking
0,biasa,no,0.0,0.0,28877,4.8,3200,0.005330,0.994670,1301,0,5,3,-1,1,1,545.0,23,41580.0
1,gold,no,0.0,0.0,103000,4.6,3100,0.006742,0.993258,2336,0,5,3,-1,1,1,653.0,48,41233.5
2,gold,no,0.0,0.0,2500,4.8,2200,0.004630,0.995370,280,0,5,3,-1,1,1,494.0,10,41571.0
3,gold,no,0.0,0.0,570000,4.8,2100,0.001577,0.998423,1417,0,5,3,-1,1,1,341.0,26,41052.5
4,gold,no,0.0,0.0,159000,4.5,2100,0.020270,0.979730,2541,0,4,3,-1,1,1,403.0,28,40821.5


In [6]:
dataset.describe()

,cashback,cashbackval,price,prodrating,reviewcount,negreview,posreview,answerscnt,otheragreemean,ratingmosthelpful,possentiment,negsentiment,sentipolarity,reviewersrep,revpictotal,prodpicstotal,ranking
count,41596.000000,41596.000000,4.159600e+04,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.0,41596.000000,41596.000000,41596.000000
mean,0.001125,228.700575,2.489824e+05,4.725113,44.338903,0.010768,0.989232,35.129363,0.002909,1.515843,3.252957,-1.064069,0.998365,1.0,10.123301,0.592557,20798.500000
std,0.006985,6191.385698,9.518654e+05,0.219873,85.406325,0.037130,0.037130,76.364433,0.056472,2.193405,0.585073,0.245956,0.045441,0.0,15.468910,1.477846,12007.630252
min,0.000000,0.000000,1.000000e+02,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,1.000000,-3.000000,-1.000000,1.0,0.000000,0.000000,86.000000
25%,0.000000,0.000000,1.600000e+04,4.600000,11.000000,0.000000,1.000000,7.000000,0.000000,0.000000,3.000000,-1.000000,1.000000,1.0,5.000000,0.000000,10221.500000
50%,0.000000,0.000000,6.000000e+04,4.800000,22.000000,0.000000,1.000000,16.000000,0.000000,0.000000,3.000000,-1.000000,1.000000,1.0,8.000000,0.000000,20768.000000
75%,0.000000,0.000000,1.699000e+05,4.900000,45.000000,0.000000,1.000000,35.000000,0.000000,4.000000,4.000000,-1.000000,1.000000,1.0,10.123301,1.000000,31214.000000
max,0.050000,687500.000000,5.500000e+07,5.000000,3200.000000,0.666667,1.000000,2541.000000,4.000000,5.000000,5.000000,-1.000000,1.000000,1.0,653.000000,48.000000,41596.000000


In [7]:
dftrain = dataset.values

In [8]:
dftrain

array([['biasa', 'no', 0.0, ..., 545.0, 23, 41580.0],
       ['gold', 'no', 0.0, ..., 653.0, 48, 41233.5],
       ['gold', 'no', 0.0, ..., 494.0, 10, 41571.0],
       ...,
       ['gold', 'no', 0.0, ..., 3.0, 1, 3058.5],
       ['biasa', 'no', 0.0, ..., 3.0, 0, 23707.0],
       ['biasa', 'no', 0.0, ..., 2.0, 0, 1051.5]], dtype=object)

In [9]:
dftrain.shape

(41596, 19)

In [10]:
# Encoding categorical data ('merchanttype' and 'topads')
labelencoder1 = LabelEncoder()
labelencoder2 = LabelEncoder()
# 'merchanttype'
dftrain[:, 0] = labelencoder1.fit_transform(dftrain[:, 0])
# 'topads'
dftrain[:, 1] = labelencoder2.fit_transform(dftrain[:, 1])
# onehotencoder for both 'merchanttype'
onehotencoder = OneHotEncoder(categorical_features=[0])
dftrain = onehotencoder.fit_transform(dftrain).toarray()

In [11]:
dftrain.shape

(41596, 21)

In [12]:
# dummy variables for merchanttype (3-1)
# avoiding dummy variable trap
dftrain = dftrain[:, 1:]

In [13]:
dftrain.shape

(41596, 20)

# feature selection

In [14]:
X = dftrain[:, :-1].copy()

In [15]:
y = dftrain[:, 19].copy()

In [16]:
X[0]
# merchanttype2, merchanttype3, topads, cashback, 
# cashbackval, price, prodrating, reviewcount, 
# negreview, posreview, answercnt, otheragreemean, 
# ratingmosthelpful, possentiment, negsentiment, sentipolarity, 
# reviewersrep, revpictotal, prodpicstotal

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  2.88770000e+04,  4.80000000e+00,  3.20000000e+03,
        5.33049041e-03,  9.94669510e-01,  1.30100000e+03,  0.00000000e+00,
        5.00000000e+00,  3.00000000e+00, -1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  5.45000000e+02,  2.30000000e+01])

In [17]:
X.shape

(41596, 19)

In [18]:
# Tree-based feature selection
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel

In [19]:
treeselect = ExtraTreesRegressor()
treeselect.fit(X, y)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
treeselect.feature_importances_

array([1.31694459e-02, 1.17246728e-04, 8.16046442e-03, 2.50222882e-03,
       2.25730562e-03, 1.97041446e-01, 4.37022579e-02, 4.57096674e-01,
       6.35020211e-03, 6.70821532e-03, 1.04010150e-01, 5.28332331e-04,
       4.13596950e-02, 2.43424072e-02, 7.10327254e-03, 3.61518022e-04,
       0.00000000e+00, 6.70589063e-02, 1.81302314e-02])

In [21]:
treemodel = SelectFromModel(treeselect, prefit=True)
X = treemodel.transform(X)

In [22]:
X.shape

(41596, 4)

In [23]:
X[0]
# price, reviewcount, answercnt, revpictotal

array([28877.,  3200.,  1301.,   545.])

In [26]:
dataset = dataset[['price', 'reviewcount', 'answerscnt', 'revpictotal', 'ranking']]

In [27]:
dftrain = dataset.values

In [28]:
sc = StandardScaler()

In [29]:
dftrain = sc.fit_transform(dftrain)

In [31]:
X = dftrain[:, :-1].copy()

In [33]:
dftrain

array([[-0.23123861, 36.94924257, 16.57690369, 34.57794654,  1.730712  ],
       [-0.15336637, 35.77835503, 30.13049662, 41.55977662,  1.701855  ],
       [-0.25894979, 25.24036717,  3.20664438, 31.28097122,  1.72996247],
       ...,
       [-0.18803555, -0.46061431, -0.3683613 , -0.46049703, -1.47741169],
       [-0.26105095, -0.46061431, -0.46002811, -0.46049703,  0.24222389],
       [ 1.04639746, -0.46061431, -0.27669448, -0.52514361, -1.64455742]])

In [34]:
y = dftrain[:, 4].copy()

In [35]:
X

array([[-0.23123861, 36.94924257, 16.57690369, 34.57794654],
       [-0.15336637, 35.77835503, 30.13049662, 41.55977662],
       [-0.25894979, 25.24036717,  3.20664438, 31.28097122],
       ...,
       [-0.18803555, -0.46061431, -0.3683613 , -0.46049703],
       [-0.26105095, -0.46061431, -0.46002811, -0.46049703],
       [ 1.04639746, -0.46061431, -0.27669448, -0.52514361]])

In [36]:
y

array([ 1.730712  ,  1.701855  ,  1.72996247, ..., -1.47741169,
        0.24222389, -1.64455742])

In [21]:
# array([-0.23123861, 36.94924257, 16.57690369, 34.57794654])
# price, reviewcount, answercnt, revpictotal

In [22]:
# Removing features with low variance
#from sklearn.feature_selection import VarianceThreshold

In [23]:
#selector = VarianceThreshold()

In [27]:
#selector.fit_transform(X)[0]
# reviewersrep dihilangkan

array([-1.63709649e+00, -4.52509671e-02, -2.48649798e-01, -1.61001634e-01,
       -3.69389560e-02, -2.31238606e-01,  3.40596826e-01,  3.69492426e+01,
       -1.46458720e-01,  1.46458720e-01,  1.65769037e+01, -5.15120936e-02,
        1.58848886e+00, -4.32356634e-01,  2.60491723e-01,  3.59761571e-02,
        3.45779465e+01,  1.51624155e+01])

In [37]:
# splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# 1 - Decision Tree

In [35]:
from sklearn.tree import DecisionTreeRegressor

In [36]:
regressor = DecisionTreeRegressor(min_samples_split=5, random_state=42, max_depth=5)

In [37]:
parameters = [{'min_samples_split': range(2, 15),
              'max_depth':[5, 6, 7, 8, 9],
              'min_samples_leaf': [4, 5, 6],
              'max_features': ['auto', 'log2', 'sqrt']}
             ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=10,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [38]:
# model fitting using grid search parameter
grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_
result = grid_search.cv_results_

Fitting 10 folds for each of 585 candidates, totalling 5850 fits
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=3 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2

[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7, total=   0.0s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=8, total=   0.0s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=8 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=8, total=   0.1s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_sampl

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.2s


[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=8, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=8 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6, total=   0.0s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=9 
[CV] max_depth=5, max_features=auto, min_samples_lea

[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=13 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=14 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=11 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=12, total=   0.0s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=9, total=   0.0s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=13, total=   0.0s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=14 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=12 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=13 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=13, total=   0.0s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4,

...

[Parallel(n_jobs=-1)]: Done 5664 tasks      | elapsed:   29.7s


[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=13, total=   0.1s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=14, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=14 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=13 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=3 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=12, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=13, total=   0.0s
[CV]  max_depth=9, max_featur

[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=8 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=6 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=5 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=6, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=6 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=4, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=8 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=3, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_lea

[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=9, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=13 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=10, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=9, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=10 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=14 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=12, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=10, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=13 
[CV] max_depth=9, max_features=sqrt, min_sa

[Parallel(n_jobs=-1)]: Done 5850 out of 5850 | elapsed:   30.7s finished


In [39]:
best_parameters

{'max_depth': 8,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 11}

In [40]:
best_estimator.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features='auto',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=11, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [41]:
# cross validation
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)

accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV] ................................................. , total=   0.1s
[CV] ................................................. , total=   0.1s
[CV] ................................................. , total=   0.1s
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


(-0.2602924542333961, 0.004192242429990074)

In [42]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 0.2504420519454316
R^2: 0.7487872929275652


In [43]:
joblib.dump(best_estimator, './training/regressor_destree_featureReduced.pkl')

['./training/regressor_destree_featureReduced.pkl']

# 2 - Linear Regression

In [44]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [45]:
# robust regression using RANSAC
from sklearn.linear_model import RANSACRegressor
ransac = RANSACRegressor(LinearRegression(),
        max_trials=300,
        min_samples=50,
        loss='absolute_loss',
        residual_threshold=5.0,
        random_state=42)
ransac.fit(X_train, y_train)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

In [46]:
# model evaluation
from sklearn.metrics import mean_squared_error, r2_score
y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)
y_train_pred_ransac = ransac.estimator_.predict(X_train)
y_test_pred_ransac = ransac.estimator_.predict(X_test)

In [47]:
def evaluate(y_train_pred, y_train, y_test_pred, y_test):
    print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
    print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

In [48]:
evaluate(y_train_pred, y_train, y_test_pred, y_test)

MSE train: 0.779, test: 0.793
R^2 train: 0.222, test: 0.205


In [49]:
evaluate(y_train_pred_ransac, y_train, y_test_pred_ransac, y_test)

MSE train: 0.811, test: 0.843
R^2 train: 0.189, test: 0.154


# 3 - SVR

In [50]:
from sklearn.svm import SVR
regressor = SVR(kernel='poly')

In [51]:
# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 5, 10, 100], 'kernel':['linear']},
        {'C':[1, 5, 10, 100], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
        ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=10,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [52]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

Fitting 10 folds for each of 24 candidates, totalling 240 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total= 1.4min
[CV] C=5, kern

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.7min


[CV] ............................... C=5, kernel=linear, total= 2.3min
[CV] C=10, kernel=linear .............................................
[CV] ............................... C=5, kernel=linear, total= 2.3min
[CV] C=100, kernel=linear ............................................
[CV] ............................... C=5, kernel=linear, total= 2.4min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total= 3.8min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total= 3.9min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total= 3.5min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total= 3.5min
[CV] C=100, kernel=linear ............................................
[CV] .

[CV] ..................... C=1, gamma=0.001, kernel=rbf, total= 1.3min
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total= 1.2min
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total= 1.3min
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total= 1.2min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total= 1.2min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total= 1.2min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total= 1.2min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] .

[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.3min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.3min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.3min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.3min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.3min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.3min
[CV] C=10, gamma=0.1, kernel=rbf .....................................


[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 50.0min


[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.3min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 1.2min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 1.2min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 1.2min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 1.1min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 1.2min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 1.2min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] .

[CV] ..................... C=100, gamma=0.5, kernel=rbf, total= 2.5min
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total= 2.4min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total= 2.5min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total= 2.5min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total= 2.6min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total= 1.6min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total= 1.6min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .

[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 74.0min finished


In [53]:
best_parameters

{'C': 100, 'gamma': 0.5, 'kernel': 'rbf'}

In [54]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)
accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total= 1.7min
[CV] ................................................. , total= 1.8min
[CV] ................................................. , total= 2.0min
[CV] ................................................. , total= 2.1min
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  2.1min remaining:  2.1min


[CV] ................................................. , total= 2.1min
[CV] ................................................. , total= 2.1min
[CV] ................................................. , total= 2.2min
[CV] ................................................. , total= 2.2min
[CV] ................................................. , total= 2.2min


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.2min finished


(-0.31353999469791377, 0.006251468552569714)

In [55]:
accuracies

array([-0.31431405, -0.32018432, -0.32254451, -0.31281012, -0.30574093,
       -0.30786898, -0.3185671 , -0.32013316, -0.30847628, -0.30476049])

In [74]:
evaluate(best_estimator, X_test, y_test=y_test)

Model Validation
MSE: 0.3059509533994428
R^2: 0.6931075806246116


In [75]:
joblib.dump(best_estimator, './training/regressor_svr_featureReduced.pkl')

['./training/regressor_svr_featureReduced.pkl']

# 4 - Multi-Layer Perceptron

In [38]:
from sklearn.neural_network import MLPRegressor
regressor = MLPRegressor(random_state=42)

In [39]:
# read csv file
dataset = pd.read_csv('./csvfiles/output_training.csv')
dataset['merchanttype'] = dataset['merchanttype'].astype('category')
dataset['merchantname'] = dataset['merchantname'].astype('category')
dataset['topads'] = dataset['topads'].replace((1,0), ('yes', 'no'))
dataset['topads'] = dataset['topads'].astype('category')
# drop column 'id' and 'prodname'
dataset = dataset.drop(['id', 'prodname', 'merchantname', 'actualrevcount'], axis=1)
# some 'revpictotal' has 9999 value and most of them have
# 'actualrevcount' = 10, only 5a986371b8a9f712ce73da7f has 119 'actualrevcount'
# replace 9999 with np.nan
dataset['revpictotal'] = dataset['revpictotal'].replace(9999, np.nan)
# data imputation
# mean technique
meanway = dataset['revpictotal'].mean()
dataset['revpictotal'] = dataset['revpictotal'].fillna(meanway)

dataset = dataset[['price', 'reviewcount', 'answerscnt', 'revpictotal', 'ranking']]
dftrain = dataset.values
sc = MinMaxScaler()
dftrain = sc.fit_transform(dftrain)
X = dftrain[:, :-1].copy()
y = dftrain[:, 4].copy()
# splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [46]:
# GRID SEARCH
parameters = [{
    'hidden_layer_sizes': [(8, ), (32,), (64,), (120,120)],
    'activation': ['logistic', 'relu'],
    'solver': ['adam', 'lbfgs'],
    'batch_size': ['auto', 8, 16, 32, 64, 128],
    'early_stopping': [True]
    } 
    ]

grid = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=10,
        n_jobs=-1,
        refit='neg_mean_squared_error',
        verbose=2
    )

In [47]:
grid_result = grid.fit(X_train, y_train)

best_parameters = grid_result.best_params_
best_estimator = grid_result.best_estimator_
result = grid_result.cv_results_

Fitting 10 folds for each of 96 candidates, totalling 960 fits
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=lbfgs 
[CV] activation=lo

/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam, total=  57.8s
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=lbfgs 


/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=adam, total=  58.4s
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(8,), solver=lbfgs 


...

[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  27.4s
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  32.2s
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  32.3s
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  43.1s
[CV] activation=relu, batch_size=64, early_stopping=True, hidd

[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.3min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.4min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=adam, total= 1.8min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=adam, total= 1.9min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=adam, total= 1.8min
[CV] activa

[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  40.6s
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  42.9s
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(120, 120), solver=lbfgs, total= 6.1min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  54.7s
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=128, early_stopp

[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.2min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(120, 120), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.3min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(120, 120), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.3min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(120, 120), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.3min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(120, 120), solver=lbfgs 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(120, 120), solver=adam, total=  34.9s
[CV] activati

[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 141.7min finished


In [48]:
evaluate(best_estimator, X_train, y_train)

Model Validation
MSE: 0.02402477877919435
R^2: 0.7130995771748272


In [49]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 0.02369225656834491
R^2: 0.715984480342131


In [51]:
# cross validation
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)

accuracies.mean(), accuracies.std() 

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total= 1.0min
[CV] ................................................. , total= 1.1min
[CV] ................................................. , total= 1.5min
[CV] ................................................. , total= 1.5min
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.5min remaining:  1.5min


[CV] ................................................. , total= 1.5min
[CV] ................................................. , total= 1.5min
[CV] ................................................. , total= 1.6min
[CV] ................................................. , total= 1.6min
[CV] ................................................. , total= 1.6min


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


(-0.024344803701130267, 0.0011549298670385255)

In [53]:
accuracies

array([-0.02601667, -0.02654906, -0.02420324, -0.02381428, -0.02276345,
       -0.02498668, -0.02419764, -0.02433579, -0.02295429, -0.02362694])

In [52]:
# save fitted model to file
joblib.dump(best_estimator, './training/regressor_mlp_featureReduced.pkl')

['./training/regressor_mlp_featureReduced.pkl']